moving forward with fusion approach, combining segmented glm and aggregate random forest
## *Perf Metrics for Explore_Expiry.ipynb*

In [1]:
ls()

character(0)

In [6]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [7]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
library(rPython)
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)
library(dominanceanalysis)

getwd()

[1] "/home/jupyter/local/Domains_202003/scripts/phaseII_05_moredata"

In [5]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')


source('../phaseII_03_forest/load_prep_data_expiry.R')

# expiry_train_prepped_2_1 (list, less df's w/ 0 obs)
# expiry_test_prepped_2_1  (list, less df's w/ 0 obs)
# expiry_train_df_1 (above, rbound)
# expiry_test_df_1 (above, rbound)
# expiry_train_df_sub (subset vars)                                        
# expiry_test_df_sub (subset vars)

# Plus
## (i.e. agg glm w/ tld & reseller, plus creation based vars as predictors)

In [13]:
# processed glm test data
filename = 'agg_glm_plus_pred'
# system(paste(sprintf("gsutil cp gs://data_input/%s /home/jupyter/local/Domains_202003/data/", filename)))
load(file.path("/home/jupyter/local/Domains_202003/data/output", filename))

In [14]:
head(agg_glm_plus_pred)

renewal_type,renewed_count,expiry_date,domain_id,domain,creation_date,status,tld,registrar,reseller,reseller_country,region,reg_period,registrant_country,renewal_status,renew_mbg,renew_type,autorenew_type,renew_date,renew_registrar,renew_reseller,reg_revenue,reg_arpt,renew_period,renew_domain_revenue,renew_arpt,reg_arpt_org,tld_registrar_index,sld,sld_type,sld_length,sld_type2,day_domains,log_reg_arpt,gibb_score,pattern,cluster,pattern_score,pattern_domain_count,reg_durY,mday,wday,qday,yday,probabilities,first_renewal_prediction
<chr>,<int>,<date>,<int>,<chr>,<date>,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<fct>,<int>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<int>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>
FirstTime,1,2020-01-22,91639803,segeln.fun,2019-01-22,Active,fun,1&1 internet,1&1 internet,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-22,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,segeln,l,6,6l,1,2.708050,2.23,segeln,1,1,1,1,22,3,22,22,0.3947892,0.395
FirstTime,1,2020-03-07,95819759,harzhotel.fun,2019-03-07,Active,fun,1&1 internet,1&1 internet,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-03-07,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,harzhotel,l,9,9l,1,2.708050,3.45,harzhotel,1,1,1,1,7,5,66,66,0.3658402,0.366
FirstTime,1,2020-01-15,91044973,movistar.fun,2019-01-15,Active,fun,10dencehispahard,10dencehispahard,Spain,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-15,10dencehispahard,10dencehispahard,15,15,1,15,15,15,fun10dencehispahard,movistar,l,8,8l,1,2.708050,8.62,movistar,1,1,1,1,15,3,15,15,0.1481456,0.148
FirstTime,1,2020-02-01,92472913,kuinik.fun,2019-02-01,Active,fun,10dencehispahard,10dencehispahard,Spain,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-02-01,10dencehispahard,10dencehispahard,15,15,1,15,15,15,fun10dencehispahard,kuinik,l,6,6l,1,2.708050,3.13,kuinik,1,1,1,1,1,6,32,32,0.1430743,0.143
FirstTime,1,2020-01-21,91532433,bulgari.fun,2019-01-21,Active,fun,1api,1api,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-21,1API,1API,5,5,1,15,15,5,fun1api,bulgari,l,7,7l,1,1.609438,4.51,bulgari,1,1,1,1,21,2,21,21,0.3700480,0.370
FirstTime,1,2020-04-24,101717674,gotsome.fun,2019-04-24,Active,fun,1api,1api,Germany,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-04-24,1API,1API,5,5,1,15,15,5,fun1api,gotsome,l,7,7l,1,1.609438,6.09,gotsome,1,1,1,1,24,4,24,114,0.3005961,0.301


In [15]:
df_plus = data.frame("actual" = agg_glm_plus_pred$renewal_status,
                     "predicted" = agg_glm_plus_pred$first_renewal_prediction)

lift_df_plus <- chart_lift(pred_df = df_plus,
                        dep_var = "actual",
                        pred_var = "predicted")

gains_plot_plus <- plot_gains (lift_df_plus) 

auc_val_plus <- calc_auc(lift_df_plus)


# saveRDS(lift_df_seg_glm_expiry_exptest, file.path('../../data/output/',"lift_df_seg_glm_expiry_exptest"))

In [16]:
auc_val_plus

[1] 0.812077

In [17]:
lift_df_plus

P,actu_renwd2,gain,lift
<dbl>,<int>,<dbl>,<dbl>
0.1,12657,0.4661020,4.661020
0.2,18058,0.6649972,3.324986
0.3,21260,0.7829129,2.609710
0.4,23335,0.8593261,2.148315
0.5,24933,0.9181734,1.836347
0.6,25985,0.9569140,1.594857
0.7,26611,0.9799669,1.399953
0.8,26970,0.9931873,1.241484
0.9,27133,0.9991898,1.110211


# Basic
## (i.e. agg glm w/ tld & reseller as predictors)

In [8]:
# processed glm test data
filename = 'agg_glm_basic_pred'
# system(paste(sprintf("gsutil cp gs://data_input/%s /home/jupyter/local/Domains_202003/data/", filename)))
load(file.path("/home/jupyter/local/Domains_202003/data/output", filename))

In [9]:
head(agg_glm_basic_pred)

renewal_type,renewed_count,expiry_date,domain_id,domain,creation_date,status,tld,registrar,reseller,reseller_country,region,reg_period,registrant_country,renewal_status,renew_mbg,renew_type,autorenew_type,renew_date,renew_registrar,renew_reseller,reg_revenue,reg_arpt,renew_period,renew_domain_revenue,renew_arpt,reg_arpt_org,tld_registrar_index,sld,sld_type,sld_length,sld_type2,day_domains,log_reg_arpt,gibb_score,pattern,cluster,pattern_score,pattern_domain_count,reg_durY,mday,wday,qday,yday,probabilities,first_renewal_prediction
<chr>,<int>,<date>,<int>,<chr>,<date>,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<fct>,<int>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<int>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>
FirstTime,1,2020-01-22,91639803,segeln.fun,2019-01-22,Active,fun,1&1 internet,1&1 internet,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-22,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,segeln,l,6,6l,1,2.708050,2.23,segeln,1,1,1,1,22,3,22,22,0.3713604,0.371
FirstTime,1,2020-03-07,95819759,harzhotel.fun,2019-03-07,Active,fun,1&1 internet,1&1 internet,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-03-07,1&1 Internet,1&1 Internet,15,15,1,15,15,15,fun1&1 internet,harzhotel,l,9,9l,1,2.708050,3.45,harzhotel,1,1,1,1,7,5,66,66,0.3604911,0.360
FirstTime,1,2020-01-15,91044973,movistar.fun,2019-01-15,Active,fun,10dencehispahard,10dencehispahard,Spain,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-01-15,10dencehispahard,10dencehispahard,15,15,1,15,15,15,fun10dencehispahard,movistar,l,8,8l,1,2.708050,8.62,movistar,1,1,1,1,15,3,15,15,0.1228685,0.123
FirstTime,1,2020-02-01,92472913,kuinik.fun,2019-02-01,Active,fun,10dencehispahard,10dencehispahard,Spain,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-02-01,10dencehispahard,10dencehispahard,15,15,1,15,15,15,fun10dencehispahard,kuinik,l,6,6l,1,2.708050,3.13,kuinik,1,1,1,1,1,6,32,32,0.1200240,0.120
FirstTime,1,2020-01-21,91532433,bulgari.fun,2019-01-21,Active,fun,1api,1api,Germany,Non China,1,NA,Renewed,0,auto-renewal,realized,2020-01-21,1API,1API,5,5,1,15,15,5,fun1api,bulgari,l,7,7l,1,1.609438,4.51,bulgari,1,1,1,1,21,2,21,21,0.3255836,0.326
FirstTime,1,2020-04-24,101717674,gotsome.fun,2019-04-24,Active,fun,1api,1api,Germany,Non China,1,NA,Not Renewd,0,auto-renewal,unrealized,2020-04-24,1API,1API,5,5,1,15,15,5,fun1api,gotsome,l,7,7l,1,1.609438,6.09,gotsome,1,1,1,1,24,4,24,114,0.3298929,0.330


In [10]:
df_basic = data.frame("actual" = agg_glm_basic_pred$renewal_status,
          "predicted" = agg_glm_basic_pred$first_renewal_prediction)

lift_df_basic <- chart_lift(pred_df = df_basic,
                        dep_var = "actual",
                        pred_var = "predicted")

gains_plot_basic <- plot_gains (lift_df_basic) 

auc_val_basic <- calc_auc(lift_df_basic)


# saveRDS(lift_df_seg_glm_expiry_exptest, file.path('../../data/output/',"lift_df_seg_glm_expiry_exptest"))

In [11]:
auc_val_basic

[1] 0.81232

In [12]:
lift_df_basic

P,actu_renwd2,gain,lift
<dbl>,<int>,<dbl>,<dbl>
0.1,12690,0.4673173,4.673173
0.2,18189,0.6698214,3.349107
0.3,21266,0.7831339,2.610446
0.4,23205,0.8545388,2.136347
0.5,24971,0.9195728,1.839146
0.6,25971,0.9563985,1.593997
0.7,26632,0.9807402,1.401057
0.8,26955,0.9926349,1.240794
0.9,27129,0.9990425,1.110047


# Var imp
https://cran.r-project.org/web/packages/dominanceanalysis/vignettes/da-logistic-regression.html

## plus

In [8]:
filename = 'agg_glm_plus_model'
# system(paste(sprintf("gsutil cp gs://data_input/%s /home/jupyter/local/Domains_202003/data/", filename)))
load(file.path("/home/jupyter/local/Domains_202003/data/output", filename))

In [ ]:
da_plus <- dominanceAnalysis(agg_glm_plus_model)
da_plus_fits <- getFits(da_plus,"r2.m")

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

In [ ]:
da_plus_fits

In [ ]:
1

In [ ]:
save(da_plus, file="../../data/output/da_plus")

In [ ]:
rm(da_plus)

In [ ]:
rm(agg_glm_plus_model)

In [ ]:
1

## basic

In [ ]:
filename = 'agg_glm_basic_model'
# system(paste(sprintf("gsutil cp gs://data_input/%s /home/jupyter/local/Domains_202003/data/", filename)))
load(file.path("/home/jupyter/local/Domains_202003/data/output", filename))

In [ ]:
da_basic <- dominanceAnalysis(agg_glm_basic_model)
da_basic_fits <- getFits(da_basic,"r2.m")

In [ ]:
save(da_basic, file="../../data/output/da_basic")

In [ ]:
rm(da_basic)

In [ ]:
rm(agg_glm_basic_model)

In [ ]:
1

# Write to GCP

In [ ]:
system("gsutil cp /home/jupyter/local/Domains_202003/data/output/* gs://data_outputt/output/")
system("mv /home/jupyter/local/Domains_202003/data/output/* /home/jupyter/local/Domains_202003/data/")